In [138]:
import docx
import PyPDF2
import pdfplumber
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

import os
import re

In [139]:
def get_files(source_path):
    for _,_,filenames in os.walk(source_path):
        for filename in filenames:
            print(filename)
            if filename.split(".")[-1] == 'pdf':
#                 continue
                handle_pdf(source_path+filename,filename)
            elif filename.split(".")[-1] == 'docx':
                handle_docx(source_path+filename,filename)

In [168]:
def handle_pdf(file_path,filename):
    """处理原始招标文件(pdf格式)"""
    pdf_obj = PyPDF2.PdfFileReader(file_path)
    doc_info = pdf_obj.documentInfo
    print(doc_info)

#     new_meta = {
#         "Author":doc_info.dc_creator,
#         "CreateDate":doc_info.xmp_createDate,
#         "ModifyDate":doc_info.xmp_modifyDate
        
#     }
    outlines = pdf_obj.getOutlines()
    pattern = '.*(技术|参数|要求).*'
    pdf_out = PyPDF2.PdfFileWriter()
    pdf_out.addMetadata(doc_info)
    sections = []
    for outline in  outlines:
        if isinstance(outline,list):
            continue
#         print(outline)
#             print(page.extractText())
        if re.match("(第.*章|第.*部分)",outline.title):
            #print(outline.title)
            sections.append(outline)
    for index,section in enumerate(sections):
        if re.match(".*(招标公告|招标邀请书).*",section.title):
            pageNumberStart = pdf_obj.getDestinationPageNumber(section)
            if isinstance(outlines[index+1],list):
                pageNumberEnd = pdf_obj.getDestinationPageNumber(sections[index+1][0])
            else:
                pageNumberEnd = pdf_obj.getDestinationPageNumber(sections[index+1])
            #print(pageNumberStart,pageNumberEnd)
            bid_date = get_metadata(file_path,filename,pageNumberStart,pageNumberEnd)
            cc = {}
            cc["BidDate"] = bid_date
            pdf_out.addMetadata(cc)
        if re.match(pattern,section.title):
            #print(section.title)
            pageNumberStart = pdf_obj.getDestinationPageNumber(section)
            if isinstance(outlines[index+1],list):
                pageNumberEnd = pdf_obj.getDestinationPageNumber(sections[index+1][0])
            else:
                pageNumberEnd = pdf_obj.getDestinationPageNumber(sections[index+1])
            #print(pageNumberStart,pageNumberEnd)
            for page in range(pageNumberStart,pageNumberEnd):
                pdf_out.addPage(pdf_obj.getPage(page))
    with open("../source/extract/"+filename,"wb") as out_file:
        try:
            pdf_out.write(out_file)
        except UnicodeEncodeError:
            print("编码错误")
            pass

In [169]:
get_files("../source/origin/")

52434f04-b69a-424a-b1c4-7e9eab615a02.pdf
{'/Title': '无标题文档', '/Author': 'NTKO', '/Creator': 'Microsoft® Word 2013', '/CreationDate': "D:20190417182942+08'00'", '/ModDate': "D:20190417182942+08'00'", '/Producer': 'Microsoft® Word 2013'}
987141ed-0940-45af-a1e5-a12f40b2a19e.pdf
{'/Title': '无标题文档', '/Author': 'NTKO', '/Creator': 'Microsoft® Word 2013', '/CreationDate': "D:20190423173053+08'00'", '/ModDate': "D:20190423173053+08'00'", '/Producer': 'Microsoft® Word 2013'}
7a58194e-13db-48d9-a741-74a17b9a56d3.pdf
{'/Title': '无标题文档', '/Author': 'NTKO', '/Creator': 'Microsoft® Word 2013', '/CreationDate': "D:20190722180514+08'00'", '/ModDate': "D:20190722180514+08'00'", '/Producer': 'Microsoft® Word 2013'}
CSIEZB170202506-采煤机-招标文件-第一册.doc
CSIE08944招标文件重新招标版-宁煤采煤机20171215.doc
7d5510a1-a1eb-42ac-9721-baa44c723bd0.pdf
{'/Title': '无标题文档', '/Author': 'NTKO', '/Creator': 'Microsoft® Word 2013', '/CreationDate': "D:20190522110918+08'00'", '/ModDate': "D:20190522110918+08'00'", '/Producer': 'Microsoft

In [ ]:
def handle_docx(file_path,filename):
    doc_obj = docx.Document(file_path)
    paragraphs = doc_obj.paragraphs
#     for table in doc_obj.tables:
#         print(table)
    sections = []
    pattern = '.*(技术|参数|要求).*'
    doc_out = docx.Document()
    for index,paragraph in enumerate(paragraphs):
        #print(paragraph.style.name)
        if "Heading" in paragraph.style.name:
            sections.append((index,paragraph))
    for order,(index,section) in enumerate(sections):
        if re.match(pattern,section.text):
#             print(paragraphs[index].text)
            #doc_out.add_paragraph(paragraphs)
            for para in paragraphs[index:sections[order+1][0]]:
                doc_out.add_paragraph(para.text,para.style)
    doc_out.save("../source/extract/"+filename)
        
#     author,created,modified,title = doc_obj.core_properties
#     print(author,created,modified,title)


In [93]:
get_files("../source/origin/")

52434f04-b69a-424a-b1c4-7e9eab615a02.pdf
987141ed-0940-45af-a1e5-a12f40b2a19e.pdf
7a58194e-13db-48d9-a741-74a17b9a56d3.pdf
CSIEZB170202506-采煤机-招标文件-第一册.doc
CSIE08944招标文件重新招标版-宁煤采煤机20171215.doc
7d5510a1-a1eb-42ac-9721-baa44c723bd0.pdf
05dd9e61-b7ce-4509-8cb6-71615f0db2bf.pdf
2fcfa91d-b082-4864-92f1-48e5aa4c4dd9.pdf
650a0471-e58b-46a9-a975-3d2ace4b9422.pdf
5336d28e-0c44-4fb0-9a08-2a185b31639a.pdf
63794ba0-37a2-4e79-a1a5-18210e95b612.pdf
89363cf8-a3e2-49b0-95c5-18fcaba047ad.pdf
00611434-ed85-42c0-a4af-59494754cbf7.pdf
b0b63dd3-7814-4d70-a218-47be7afd1bcf.docx
国际招标第一册范本-中文版.pdf
a9a8b519-fb52-450c-bdda-444ad648618b.pdf
ae0edeff-1f0a-4191-99f1-f53ccdbc675e.pdf
22515c19-81aa-4f21-b82f-3c1e2c82c009.doc


In [ ]:
origin = pdfplumber.open("../source/extract/2fcfa91d-b082-4864-92f1-48e5aa4c4dd9.pdf")
origin.metadata

In [ ]:
def get_metadata(file_path,filename,pageNumberStart,pageNumberEnd):
    origin = pdfplumber.open(file_path)
    pages = origin.pages[pageNumberStart:pageNumberEnd]
    pattern = ".*(开标时间|投标截止时间).*"
    res = []
    for page in pages:
        texts = page.extract_text().split('\n')
        for index,text in enumerate(texts):
#             print(text)
            if re.match(pattern,text):
                res.append(text)
                res.append(texts[index+1])
    return "".join(res)

{'Producer': 'Microsoft® Word 2013',
 'Title': '无标题文档',
 'Author': 'NTKO',
 'Creator': 'Microsoft® Word 2013',
 'CreationDate': "D:20190522112531+08'00'",
 'ModDate': "D:20190522112531+08'00'",
 /b'BidDate': '        6.1 投标文件递交的截止时间及开标时间（投标截止时间，下同）为 2019-06-12 09:30（北京时间）。 '}

In [137]:
def get_metadata(file_path,filename,pageNumberStart,pageNumberEnd):
    origin = pdfplumber.open(file_path)
    pages = origin.pages[pageNumberStart:pageNumberEnd]
    pattern = ".*(开标时间|投标截止时间).*"
    res = []
    for page in pages:
        texts = page.extract_text().split('\n')
        for index,text in enumerate(texts):
#             print(text)
            if re.match(pattern,text):
                res.append(text)
                res.append(texts[index+1])
    return "".join(res)